<a href="https://colab.research.google.com/github/codinglover-Noman/Low-resource-language-in-NMT-/blob/main/Translator_Bn_Zh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install transformers sentencepiece datasets
# !pip install ipython

from datasets import load_dataset
# from google.colab import drive
from IPython.display import display
# from IPython.html import widgets
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
from torch import optim
from torch.nn import functional as F
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm_notebook

sns.set()

In [ ]:
model_repo = "google/mt5-small"
# model_path = "mt5_translation.pt"
max_seq_len = 20

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_repo)

C:\Users\MSI\anaconda3\lib\site-packages\transformers\convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_repo)
model = model.cpu()

In [ ]:
token_ids = tokenizer.encode(
    ' This will be translated to Japanese! (hopefully)',
    return_tensors='pt').cpu()
print(token_ids)

model_out = model.generate(token_ids)
print(model_out)

output_text = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(model_out[0]))
print(output_text)

tensor([[  1494,    898,    390,  37194,    285,    288,  30865,    309,    274,
         116024,  11994,    271,      1]])


C:\Users\MSI\anaconda3\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[     0, 250099,      1]])
<pad> <extra_id_0></s>


In [ ]:
example_input_str = ' This is just a test nbuig.'
# example_input_str = 'これは普通のテスト'
input_ids = tokenizer.encode(example_input_str, return_tensors='pt')
print('Input IDs:', input_ids)

tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
print('Tokens:', tokens)

Input IDs: tensor([[1494,  339, 1627,  259,  262, 2978,  259,  272, 1982, 1315,  260,    1]])
Tokens: ['▁This', '▁is', '▁just', '▁', 'a', '▁test', '▁', 'n', 'bu', 'ig', '.', '</s>']


In [ ]:
sorted(tokenizer.vocab.items(), key=lambda x: x[1])

[('<pad>', 0),
 ('</s>', 1),
 ('<unk>', 2),
 ('<0x00>', 3),
 ('<0x01>', 4),
 ('<0x02>', 5),
 ('<0x03>', 6),
 ('<0x04>', 7),
 ('<0x05>', 8),
 ('<0x06>', 9),
 ('<0x07>', 10),
 ('<0x08>', 11),
 ('<0x09>', 12),
 ('<0x0A>', 13),
 ('<0x0B>', 14),
 ('<0x0C>', 15),
 ('<0x0D>', 16),
 ('<0x0E>', 17),
 ('<0x0F>', 18),
 ('<0x10>', 19),
 ('<0x11>', 20),
 ('<0x12>', 21),
 ('<0x13>', 22),
 ('<0x14>', 23),
 ('<0x15>', 24),
 ('<0x16>', 25),
 ('<0x17>', 26),
 ('<0x18>', 27),
 ('<0x19>', 28),
 ('<0x1A>', 29),
 ('<0x1B>', 30),
 ('<0x1C>', 31),
 ('<0x1D>', 32),
 ('<0x1E>', 33),
 ('<0x1F>', 34),
 ('<0x20>', 35),
 ('<0x21>', 36),
 ('<0x22>', 37),
 ('<0x23>', 38),
 ('<0x24>', 39),
 ('<0x25>', 40),
 ('<0x26>', 41),
 ('<0x27>', 42),
 ('<0x28>', 43),
 ('<0x29>', 44),
 ('<0x2A>', 45),
 ('<0x2B>', 46),
 ('<0x2C>', 47),
 ('<0x2D>', 48),
 ('<0x2E>', 49),
 ('<0x2F>', 50),
 ('<0x30>', 51),
 ('<0x31>', 52),
 ('<0x32>', 53),
 ('<0x33>', 54),
 ('<0x34>', 55),
 ('<0x35>', 56),
 ('<0x36>', 57),
 ('<0x37>', 58),
 ('<0x38>',

In [ ]:
# Source: https://huggingface.co/datasets/alt
dataset = load_dataset('alt')

No config specified, defaulting to: alt/alt-parallel
Found cached dataset alt (C:/Users/MSI/.cache/huggingface/datasets/alt/alt-parallel/1.0.0/b498af66d7e1b78d98b7557e7b0b06ae54fc60491dccdc1a0f8316e677d1ce75)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
train_dataset[0]

{'SNT.URLID': '80188',
 'SNT.URLID.SNTID': '1',
 'url': 'http://en.wikinews.org/wiki/2007_Rugby_World_Cup:_Italy_31_-_5_Portugal',
 'translation': {'bg': 'ফ্রান্সের প্যারিসের পার্ক দি প্রিন্সেস-এ হওয়া ২০০৭-এর রাগবি বিশ্বকাপের পুল সি-তে ইটালি পর্তুগালকে ৩১-৫ গোলে হারিয়েছে।',
  'en': 'Italy have defeated Portugal 31-5 in Pool C of the 2007 Rugby World Cup at Parc des Princes, Paris, France.',
  'en_tok': 'Italy have defeated Portugal 31-5 in Pool C of the 2007 Rugby World Cup at Parc des Princes , Paris , France .',
  'fil': 'Natalo ng Italya ang Portugal sa puntos na 31-5 sa Grupong C noong 2007 sa Pandaigdigang laro ng Ragbi sa Parc des Princes, Paris, France.',
  'hi': '2007 में फ़्रांस, पेरिस के पार्क डेस प्रिंसेस में हुए रग्बी विश्व कप के पूल C में इटली ने पुर्तगाल को 31-5 से हराया।',
  'id': 'Italia berhasil mengalahkan Portugal 31-5 di grup C dalam Piala Dunia Rugby 2007 di Parc des Princes, Paris, Perancis.',
  'ja': 'フランスのパリ、パルク・デ・プランスで行われた2007年ラグビーワールドカップのプールCで、イタリアは31対5でポルトガルを

In [ ]:
LANG_TOKEN_MAPPING = {
    'zh':'<en>',
    'bg':'<bg>',
    'hi':'<hi>'
}

In [ ]:
special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(250103, 512)

In [ ]:
token_ids = tokenizer.encode(
    example_input_str, return_tensors='pt', padding='max_length',
    truncation=True, max_length=max_seq_len)
print(token_ids)

tokens = tokenizer.convert_ids_to_tokens(token_ids[0])
print(tokens)

tensor([[1494,  339, 1627,  259,  262, 2978,  259,  272, 1982, 1315,  260,    1,
            0,    0,    0,    0,    0,    0,    0,    0]])
['▁This', '▁is', '▁just', '▁', 'a', '▁test', '▁', 'n', 'bu', 'ig', '.', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [ ]:
def encode_input_str(text, target_lang, tokenizer, seq_len,
                     lang_token_map=LANG_TOKEN_MAPPING):
  target_lang_token = lang_token_map[target_lang]

  # Tokenize and add special tokens
  input_ids = tokenizer.encode(
      text = target_lang_token + text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]

def encode_target_str(text, tokenizer, seq_len,
                      lang_token_map=LANG_TOKEN_MAPPING):
  token_ids = tokenizer.encode(
      text = text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return token_ids[0]

def format_translation_data(translations, lang_token_map,
                            tokenizer, seq_len=128):
  # Choose a random 2 languages for in i/o
  langs = list(lang_token_map.keys())
  input_lang, target_lang = np.random.choice(langs, size=2, replace=False)

  # Get the translations for the batch
  input_text = translations[input_lang]
  target_text = translations[target_lang]

  if input_text is None or target_text is None:
    return None

  input_token_ids = encode_input_str(
      input_text, target_lang, tokenizer, seq_len, lang_token_map)

  target_token_ids = encode_target_str(
      target_text, tokenizer, seq_len, lang_token_map)

  return input_token_ids, target_token_ids

def transform_batch(batch, lang_token_map, tokenizer):
  inputs = []
  targets = []
  for translation_set in batch['translation']:
    formatted_data = format_translation_data(
        translation_set, lang_token_map, tokenizer, max_seq_len)

    if formatted_data is None:
      continue

    input_ids, target_ids = formatted_data
    inputs.append(input_ids.unsqueeze(0))
    targets.append(target_ids.unsqueeze(0))

  batch_input_ids = torch.cat(inputs).cpu()
  batch_target_ids = torch.cat(targets).cpu()

  return batch_input_ids, batch_target_ids

def get_data_generator(dataset, lang_token_map, tokenizer, batch_size=32):
  dataset = dataset.shuffle()
  for i in range(0, len(dataset), batch_size):
    raw_batch = dataset[i:i+batch_size]
    yield transform_batch(raw_batch, lang_token_map, tokenizer)


In [ ]:
# Testing `data_transform`
in_ids, out_ids = format_translation_data(
    train_dataset[0]['translation'], LANG_TOKEN_MAPPING, tokenizer)

print(' '.join(tokenizer.convert_ids_to_tokens(in_ids)))
print(' '.join(tokenizer.convert_ids_to_tokens(out_ids)))

# Testing data generator
data_gen = get_data_generator(train_dataset, LANG_TOKEN_MAPPING, tokenizer, 8)
data_batch = next(data_gen)
print('Input shape:', data_batch[0].shape)
print('Output shape:', data_batch[1].shape)

<hi> ▁ 意大利 在 法国 巴黎 王子 公园 体育 场 举办 的 2007 年 橄 榄 球 世界杯 C 组 以 3 1-5 击 败 葡萄 牙 。 </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
▁2007 ▁में ▁फ़ ्रा ंस , ▁पे रिस ▁के ▁पार् क ▁डे स ▁ प्रि ंस ेस ▁में ▁हु ए ▁ र ग् बी ▁विश्व ▁कप ▁के ▁पू ल ▁C ▁में ▁ इट ली ▁ने ▁पुर् त गाल ▁को ▁3 1-5 ▁से ▁हर ाया । </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

In [ ]:
# model.load_state_dict(torch.load(model_path))

In [ ]:
n_epochs = 3
batch_size = 16
print_freq = 50
lr = 5e-4
n_batches = int(np.ceil(len(train_dataset) / batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(total_steps * 0.01)

In [ ]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer, n_warmup_steps, total_steps)

C:\Users\MSI\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
losses = []

In [ ]:
def eval_model(model, gdataset, max_iters=8):
  test_generator = get_data_generator(gdataset, LANG_TOKEN_MAPPING,
                                      tokenizer, batch_size)
  eval_losses = []
  for i, (input_batch, label_batch) in enumerate(test_generator):
    if i >= max_iters:
      break

    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)
    eval_losses.append(model_out.loss.item())

  return np.mean(eval_losses)


In [ ]:
test_loss = eval_model(model, test_dataset)

In [ ]:
test_loss

26.336649417877197

In [ ]:
for epoch_idx in range(n_epochs):
  # Randomize data order
  data_generator = get_data_generator(train_dataset, LANG_TOKEN_MAPPING,
                                      tokenizer, batch_size)

  for batch_idx, (input_batch, label_batch) \
      in tqdm_notebook(enumerate(data_generator), total=n_batches):

    optimizer.zero_grad()

    # Forward pass
    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)

    loss = model_out.loss
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Print training update info
    if (batch_idx + 1) % print_freq == 0:
      avg_loss = np.mean(losses[-print_freq:])
      print('Epoch: {} | Step: {} | Avg. loss: {:.3f} | lr: {}'.format(
          epoch_idx+1, batch_idx+1, avg_loss, scheduler.get_last_lr()[0]))

  test_loss = eval_model(model, test_dataset)
  print('Test loss of {:.3f}'.format(test_loss))

C:\Users\MSI\AppData\Local\Temp\ipykernel_10188\4134151420.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  in tqdm_notebook(enumerate(data_generator), total=n_batches):


  0%|          | 0/1131 [00:00<?, ?it/s]

Epoch: 1 | Step: 50 | Avg. loss: 11.617 | lr: 0.0004974702380952382
Epoch: 1 | Step: 100 | Avg. loss: 5.534 | lr: 0.0004900297619047619
Epoch: 1 | Step: 150 | Avg. loss: 4.638 | lr: 0.00048258928571428574
Epoch: 1 | Step: 200 | Avg. loss: 4.233 | lr: 0.00047514880952380953
Epoch: 1 | Step: 250 | Avg. loss: 4.079 | lr: 0.0004677083333333333
Epoch: 1 | Step: 300 | Avg. loss: 3.973 | lr: 0.0004602678571428571
Epoch: 1 | Step: 350 | Avg. loss: 3.784 | lr: 0.00045282738095238096
Epoch: 1 | Step: 400 | Avg. loss: 3.697 | lr: 0.0004453869047619048
Epoch: 1 | Step: 450 | Avg. loss: 3.557 | lr: 0.0004379464285714286
Epoch: 1 | Step: 500 | Avg. loss: 3.545 | lr: 0.0004305059523809524
Epoch: 1 | Step: 550 | Avg. loss: 3.458 | lr: 0.0004230654761904762
Epoch: 1 | Step: 600 | Avg. loss: 3.514 | lr: 0.00041562500000000003
Epoch: 1 | Step: 650 | Avg. loss: 3.410 | lr: 0.0004081845238095238
Epoch: 1 | Step: 700 | Avg. loss: 3.429 | lr: 0.00040074404761904767
Epoch: 1 | Step: 750 | Avg. loss: 3.282 | l

  0%|          | 0/1131 [00:00<?, ?it/s]

Epoch: 2 | Step: 50 | Avg. loss: 3.124 | lr: 0.0003291666666666667
Epoch: 2 | Step: 100 | Avg. loss: 3.112 | lr: 0.00032172619047619053
Epoch: 2 | Step: 150 | Avg. loss: 3.097 | lr: 0.00031428571428571427
Epoch: 2 | Step: 200 | Avg. loss: 3.077 | lr: 0.0003068452380952381
Epoch: 2 | Step: 250 | Avg. loss: 3.115 | lr: 0.0002994047619047619
Epoch: 2 | Step: 300 | Avg. loss: 3.082 | lr: 0.00029196428571428575
Epoch: 2 | Step: 350 | Avg. loss: 3.036 | lr: 0.00028452380952380954
Epoch: 2 | Step: 400 | Avg. loss: 3.129 | lr: 0.00027708333333333334
Epoch: 2 | Step: 450 | Avg. loss: 3.022 | lr: 0.00026964285714285713
Epoch: 2 | Step: 500 | Avg. loss: 3.037 | lr: 0.000262202380952381
Epoch: 2 | Step: 550 | Avg. loss: 3.002 | lr: 0.00025476190476190477
Epoch: 2 | Step: 600 | Avg. loss: 3.075 | lr: 0.00024732142857142856
Epoch: 2 | Step: 650 | Avg. loss: 2.966 | lr: 0.00023988095238095238
Epoch: 2 | Step: 700 | Avg. loss: 3.055 | lr: 0.0002324404761904762
Epoch: 2 | Step: 750 | Avg. loss: 2.973 |

  0%|          | 0/1131 [00:00<?, ?it/s]

Epoch: 3 | Step: 50 | Avg. loss: 2.843 | lr: 0.00016086309523809526
Epoch: 3 | Step: 100 | Avg. loss: 2.935 | lr: 0.00015342261904761906
Epoch: 3 | Step: 150 | Avg. loss: 2.853 | lr: 0.00014598214285714288
Epoch: 3 | Step: 200 | Avg. loss: 2.856 | lr: 0.00013854166666666667
Epoch: 3 | Step: 250 | Avg. loss: 2.906 | lr: 0.0001311011904761905
Epoch: 3 | Step: 300 | Avg. loss: 2.889 | lr: 0.00012366071428571428
Epoch: 3 | Step: 350 | Avg. loss: 2.927 | lr: 0.0001162202380952381
Epoch: 3 | Step: 400 | Avg. loss: 2.871 | lr: 0.0001087797619047619
Epoch: 3 | Step: 450 | Avg. loss: 2.873 | lr: 0.00010133928571428571
Epoch: 3 | Step: 500 | Avg. loss: 2.853 | lr: 9.389880952380952e-05
Epoch: 3 | Step: 550 | Avg. loss: 2.881 | lr: 8.645833333333334e-05
Epoch: 3 | Step: 600 | Avg. loss: 2.734 | lr: 7.901785714285714e-05
Epoch: 3 | Step: 650 | Avg. loss: 2.864 | lr: 7.157738095238095e-05
Epoch: 3 | Step: 700 | Avg. loss: 2.867 | lr: 6.413690476190476e-05
Epoch: 3 | Step: 750 | Avg. loss: 2.919 | l

## Manual Testing

In [ ]:
saved_model_filepath = "Saved Model/Translator_Bn_Zh.pth"

torch.save(model.state_dict(), saved_model_filepath)
# Load the saved best model state_dict()
model.load_state_dict(torch.load(saved_model_filepath))

<All keys matched successfully>

In [ ]:
# test_sentence = test_dataset[2]['translation']['zh']
test_sentence = "我喜欢学中文"
print('Raw input text:', test_sentence)

input_ids = encode_input_str(
    text = test_sentence,
    target_lang = 'bg',
    tokenizer = tokenizer,
    seq_len = model.config.max_length,
    lang_token_map = LANG_TOKEN_MAPPING)
input_ids = input_ids.unsqueeze(0).cpu()

print('Truncated input text:', tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[0])))

Raw input text: 我喜欢学中文
Truncated input text: <bg> 我喜欢学中文</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [ ]:
output_tokens = model.generate(input_ids, num_beams=10, num_return_sequences=3)
# print(output_tokens)
for token_set in output_tokens:
  print(tokenizer.decode(token_set, skip_special_tokens=True))

আমি খুব ভালো বলেছি যে, আমি ভালো বলেছি যে আমি
আমি খুব ভালো বলেছি যে, আমি ভালো বলেছি যে, 
আমি খুব ভালো বলেছি যে, আমি ইংরেজি ভাষা


## TEST OUTPUT

In [ ]:

input_text = '' # {type:"string"}
output_language = 'hi' # ["bg", "hi", "zh"]


input_ids = encode_input_str(
    text = input_text,
    target_lang = output_language,
    tokenizer = tokenizer,
    seq_len = model.config.max_length,
    lang_token_map = LANG_TOKEN_MAPPING)
input_ids = input_ids.unsqueeze(0).cpu()

output_tokens = model.generate(input_ids, num_beams=20, length_penalty=0.2)
print(input_text + '  ->  ' + \
      tokenizer.decode(output_tokens[0], skip_special_tokens=True))

我是学生  ->  उन्होंने यह भी बताया कि मैंने अपने विद्यार्थियों के 
